# Siamese network implementation with the strips dataset

With denoised and averaged images

## Dataset structure

The images themselves are stored already cut as strips, with the characteristic extracted by denoising them and getting the residual, as described in Lukas et al. We have an indexing file to retrieve them (info data) in the following structure:

* Info data:
    * Student 0:
        * Image 0:
            * Strip 0,0
            * Strip 0,256
            * Strip 0,512
            * ...
        * Image 1:
            * Strip 0,0
            * ...
        * ...
    * Student 1:
        * ...

Strip naming convention:

* Eurecom_NNN_picXG_III_XXXX_YYYY.PNG -> Strip from:
    * position (XXXX,YYYY) (top, left), from: 
    * image picXG_III, from:
    * Student with id NNN

Then, the function get_batch generates a random sample from the dataset. It allows to draw either from the training or validation split of the dataset (defined at the level of the students, 0.8 of the students are in train and the rest in validation).
The generated batch has the following shape:

* (label, (batch_size, examples, height, width, channels), (batch_size, examples, height, width, channels))  Since images are grayscale, in fact channels should be 1

More clearly:

* batch:
    * label 0:
        * strip_x-y from Image j from Student i  <---------->  strip_w-t from Image k from Student l
        * strip_x-y from Image j2 from Student i  <---------->  strip_w-t from Image k2 from Student l
        * strip_x-y from Image j3 from Student i  <---------->  strip_w-t from Image k3 from Student l
        * strip_x-y from Image j4 from Student i  <---------->  strip_w-t from Image k4 from Student l
    * label 0:
        * ...
        
    * label 1:
        * strip_x-y from Image j from Student i  <---------->  strip_w-t from Image k from Student i
        * strip_x-y from Image j2 from Student i  <---------->  strip_w-t from Image k2 from Student i
        * strip_x-y from Image j3 from Student i  <---------->  strip_w-t from Image k3 from Student i
        * strip_x-y from Image j4 from Student i  <---------->  strip_w-t from Image k4 from Student i
    * label 1:
        * ...
        
Note that ${i \neq l, j \neq j_i}$ etc. 

            
            

## Training pipeline

During training, a batch is generated at random from the training slice, ensuring the previously described invariant. The next steps are as follows:

* Images are normalized
* We define the left and right input, according to the tuple in the pair (for label == 0, all strips belong to the same camera and position, while for label == 1, each input belongs to a different camera.)
* All images from each input are averaged element-wise (separetely, of course)
* The result is fed, one at a time to a CNN model defined at sequential_block. Note that the weights are the same for each input
* The CNN outputs a feature vector of size 2048
* The output from each of the inputs is compared by calculating the absolute distance between the two vectors
* This result is fed into a final Dense layer, for classification, with sigmoid activation. 
    * A result close to 0 indicates that the strips from each input belong to different devices.
    * A result close to 1 indicates that the strips from each input where taken by the same device.


In [2]:
import os
import numpy as np
import random as rng
import cv2
import json

In [3]:
info_data = []
with open('/home/data/strips_socrates/dataset_info.json') as json_file: 
    info_data = json.load(json_file) 

In [4]:
STRIP_SIZE = 256
SUB_STRIP_SIZE = 128

In [69]:
def get_batch(info_data, batch_size, examples, with_id=False, dataset = 'train'):
    pairs = [np.zeros((batch_size, examples, STRIP_SIZE, STRIP_SIZE)) for i in range(2)]
    labels = np.zeros((batch_size, ))
    labels[batch_size//2:] = 1
    split_index = int((len(info_data)-1) * 0.8)
    if dataset == 'train':
        students = [rng.randint(0, split_index) for _ in range(batch_size)] 
    elif dataset == 'whole':
        students = [rng.randint(0, (len(info_data)-1)) for _ in range(batch_size)] 
    else:
        students = [rng.randint(split_index+1, len(info_data)-1) for _ in range(batch_size)] 
    strips_loc = [rng.randint(0, len(info_data[i][0][1])-1) for i in students]
    id_pairs = []
    for i in range(batch_size):
        std = students[i]
        avl_images = [j for j in range(len(info_data[std]))]
        exl = rng.sample(avl_images, examples)
        avl_images = [j for j in avl_images if j not in exl]
        srl = strips_loc[i]
        strips1 = [ info_data[std][e][1][srl] for e in exl ]
        strips2 = []
        if i >= batch_size // 2:
            exl2 = rng.sample(avl_images, examples)
            strips2 = [info_data[std][e][1][srl] for e in exl2]
        else:
            std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
            srl2 = rng.randint(0, len(info_data[std2][0][1])-1)
            avl_images2 = [j for j in range(len(info_data[std2]))]
            exl2 = rng.sample(avl_images2, examples)
            strips2 = [ info_data[std2][e][1][srl2] for e in exl2]
        for k in range(examples):
            pairs[0][i,k,:,:] = cv2.imread(strips1[k], cv2.IMREAD_UNCHANGED)/255
            pairs[1][i,k,:,:] = cv2.imread(strips2[k], cv2.IMREAD_UNCHANGED)/255
        id_pairs.append((strips1, strips2))
    if with_id:
        return pairs, labels, id_pairs
    else:
        return pairs, labels

In [70]:
p,l,ids = get_batch(info_data, 10, 4, True)

In [72]:
for r,s in zip(l, ids):
    print(r)
    for a in s:
        for b in a:
            print(b)

0.0
/home/data/strips_socrates/108/Eurecom_108_picBG_019_256_1792.PNG
/home/data/strips_socrates/108/Eurecom_108_picBG_034_256_1792.PNG
/home/data/strips_socrates/108/Eurecom_108_picBG_018_256_1792.PNG
/home/data/strips_socrates/108/Eurecom_108_picFG_040_256_1792.PNG
/home/data/strips_socrates/130/Eurecom_130_picFG_004_2560_1792.PNG
/home/data/strips_socrates/130/Eurecom_130_picFG_037_2560_1792.PNG
/home/data/strips_socrates/130/Eurecom_130_picBG_022_2560_1792.PNG
/home/data/strips_socrates/130/Eurecom_130_picBG_030_2560_1792.PNG
0.0
/home/data/strips_socrates/135/Eurecom_135_picBG_042_2304_512.PNG
/home/data/strips_socrates/135/Eurecom_135_picBG_023_2304_512.PNG
/home/data/strips_socrates/135/Eurecom_135_picBG_032_2304_512.PNG
/home/data/strips_socrates/135/Eurecom_135_picFG_031_2304_512.PNG
/home/data/strips_socrates/123/Eurecom_123_picBG_028_2304_768.PNG
/home/data/strips_socrates/123/Eurecom_123_picBG_021_2304_768.PNG
/home/data/strips_socrates/123/Eurecom_123_picFG_036_2304_768.PN

In [8]:
def generate(info_data, examples, batch_size, dataset='train'):
    while True:
        pairs, labels = get_batch(info_data, batch_size, examples, False, dataset=dataset)
        yield(pairs, labels)

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [10]:
def sequential_block(input_shape = (256,256, 1), base_filters=64):
    model = keras.Sequential()
    model.add(layers.Conv2D(base_filters, (3,3), input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Conv2D(base_filters*4, (3,3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Conv2D(base_filters*4, (5,5)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Conv2D(base_filters*4, (5,5)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Conv2D(base_filters*4, (7,7)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(base_filters*64, activation='relu'))
    
    
    
    return model

In [11]:
seq = sequential_block(base_filters=32)

In [12]:
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 127, 127, 32)      224       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 128)     36992     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 62, 62, 128)       896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 58, 128)       4

In [13]:
def get_siamese_model(input_shape = (4, 256,256,1), base_filters=16):

    left_input = layers.Input(input_shape)
    right_input = layers.Input(input_shape)
    
    # Convolutional Neural Network
    subshape = input_shape[1:4]
    model = sequential_block(subshape, base_filters)
    
    #average noise patterns for image examples
    avged_input_l = layers.Average()([left_input[:,k,...] for k in range(input_shape[0])])
    avged_input_r = layers.Average()([right_input[:,k,...] for k in range(input_shape[0])])

    encoded_l = model(avged_input_l)
    encoded_r = model(avged_input_r)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = layers.Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = layers.Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = keras.Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [14]:
model = get_siamese_model(base_filters=40)
model.summary(200)

Model: "model"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_1 (InputLayer)                                              [(None, 4, 256, 256, 1)]                    0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
input_2 (InputLayer)                                              [(None, 4, 256, 256, 1)]                    0                                                                      

In [62]:
initial_learning_rate = 1e-6
end_learning_rate = 1e-8
decay_steps = 12800 * 5
lr_schedule = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate, decay_steps, end_learning_rate, power=1
)
model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=[keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall'), "accuracy"] 
    )
run_name = "denoised04-dropout-es02"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs/"+run_name)

checkpoint_filepath = "checkPts/siamese_"+run_name+".h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True)

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1)

In [63]:
batch_size = 16
epochs = 500
examples = 4
steps_epoch = 128

In [ ]:
model.fit(generate(info_data, examples, batch_size), 
         epochs = epochs,
         steps_per_epoch= steps_epoch,
          callbacks=[tensorboard_callback,
                     model_checkpoint_callback,
                     early_stopping_callback],
          validation_batch_size=16,
          validation_data=val_set,
         shuffle = True)

In [ ]:
model.save("last-hack02.h5")

In [65]:
model.load_weights("checkPts/siamese_denoised04-dropout-es.h5") #best model so far

### Validation:
Simple, just feed pairs from the validation split and see the performance.

In [66]:
val_set = get_batch(info_data, 500, 4, False, 'validation')

In [67]:
preds = model.predict(val_set[0])

In [ ]:
corr = [y == (p>0) for y, p in zip (val_set_y, preds)]

In [ ]:
round(preds[0][0])

In [68]:
cs = np.zeros((2,2))
for y,p in zip (val_set[1], preds):
    cs[int(y), round(p[0])] += 1 
acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)

[[228.  22.]
 [ 72. 178.]] 0.812 0.712


### Extended validation: (Not impemented for this model yet)
Take two images, and compare strip by strip, then do a majority voting.

In [ ]:
def get_val_batch(info_data, num_whole_images, batch_size):
    images = []
    split_index = int((len(info_data)-1) * 0.8)
    students = [rng.randint(split_index+1, len(info_data)-1) for _ in range(num_whole_images)] 
    labels = np.zeros((num_whole_images, ))
    labels[num_whole_images//2:] = 1
    id_pairs_img = []
    for j in range(num_whole_images):
        pairs = [np.zeros((batch_size, STRIP_SIZE, STRIP_SIZE, 3)) for i in range(2)]
        imgs = [rng.randint(0, len(info_data[i])-1) for i in students]
        std = students[j]
        img = imgs[j]
        id_pairs = []
        std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
        if j >= num_whole_images // 2:   
            img2 = (img + rng.randint(1, len(info_data[std])-1)) % len(info_data[std])
            std2 = std
        else:
            std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
            img2 = rng.randint(0, len(info_data[std2])-1)
        if batch_size > len(info_data[std][img][1]):
            print("a", info_data[std][0][0])
            std = std+1
            
        if batch_size > len(info_data[std2][img2][1]):
            print("b", info_data[std2][0][0])
            std2 = std+1
            
        for i in range(batch_size):    
            strip1 = info_data[std][img][1][i]
            strip2 = info_data[std2][img2][1][i]
#             print(strip1, strip2)
            pairs[0][i,:,:,:] = cv2.imread(strip1)/255
            pairs[1][i,:,:,:] = cv2.imread(strip2)/255
            id_pairs.append((strip1, strip2))
        images.append(pairs)
        id_pairs_img.append(id_pairs)
    
    return images, labels, id_pairs_img

In [ ]:
t_i, t_l, t_d = get_val_batch(info_data, 100, 10)

In [ ]:
cs = np.zeros((2,2))
cs_all = np.zeros((2,2))
for i, l, d in zip(t_i, t_l, t_d):
    preds = model.predict(i)
    for p in preds:
        cs_all[int(l), round(p[0])] += 1
#     print(int(l), round(np.mean(preds)), (np.mean(preds)))
    fp = sum([p[0] > 0.5 for p in preds]) > len(preds)//2 
    cs[int(l), int(fp)] +=1
#     print(l, d[0])

acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)
cs = cs_all
acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)